# zilean Demo

In this demo, you can explore how to fetch data using the `TimelineCrawler` class. You then can explore how to manipulate data using the `SnapShots` class. 

To fetch data from the Riot Games API, you need your api key. It is free to get one if you have a Riot account, just visit the [Riot Developer Portal](https://developer.riotgames.com/) and log in. 

If you don't have a API Key, don't worry! We had prepared some data for you to play around. 

In [1]:
# Run this cell
from zilean import SnapShots, TimelineCrawler, read_api_key
import pandas as pd
import json

In [2]:
# !!DON'T RUN THIS CELL IF YOU DON'T HAVE AN API KEY!!

# Insert your api key here, replacing None
api_key = read_api_key(api_key=None)

In [3]:
# RUN THIS CELL IF YOU DON'T HAVE AN API KEY
if not api_key:
    FILE = "data.json"

## Using the TimelineCrawler

`TimelineCrawler` automatically help you crawl matches with your api key. You can customize the crawler to fetch matches based on your specification.

You can choose the **region** from these options:

    ['br1', 'eun1', 'euw1', 'jp1', 'kr', 'la1', 'la2', 'na1', 'oc1', 'ru', 'tr1']

You can choose the **tier** from these options:

    ['CHALLENGER','GRANDMASTER','MASTER','DIAMOND','PLATINUM','GOLD','SILVER','BRONZE','IRON'] 

You can choose the **queue** from these options:   

    ['RANKED_SOLO_5x5', 'RANKED_FLEX_SR', 'RANKED_FLEX_TT']

## Demo

In this demo, we will crawl 50 diamond ranked solo matches from the Korean server.

We can also specify the `cutoff` of our matches. We can only keep matches that lasted longer than a specific cutoff time. In this demo, the default value is 16 (in minutes).

Crawling 50 matches will usually take less than 1 minutes. 

_If you don't have a api key, the demo file will have 10 matches_.

In [4]:
# Using the api key
if api_key:
    crawler = TimelineCrawler(api_key, region="kr", tier="DIAMOND", queue="RANKED_SOLO_5x5")
    result = crawler.crawl(50, cutoff=16)
# No api key, load the local file
else:
    with open(FILE, "r") as f:
        result = json.load(f)

Crawling matches: 100%|██████████| 50/50 [00:34<00:00,  1.43it/s]
50it [00:01, 47.67it/s]


There are in total 48 crawled matches longer than 16 minutes.


Let's see the matchid of the first crawled match.

In [5]:
print("First match id:", result[0]['metadata']['matchId'])

First match id: KR_6004304384


## Using the SnapShots class

You can take a look at `result`, which is a list of `MatchTimelineDto`s. It is a very large and messy dictionary. We can ectract core player data of the matches by using the `SnapShots` class.

One of the most important feature of the `SnapShots` class is that you can specify which **frame** to look at. A **frame** is a snapshot of a match, in minutes. For example, the 15th frame is the game stats at the 15 minute mark of the game. With the `SnapShots` class, you can specify the frames you want to look at for all the matches you have crawled. 

In this demo, we will look at the **10 and 15** minute mark of every match.

In [7]:
snaps = SnapShots(result, frames=[10, 15])

We can see the statistics of the matches at 10 and 15 minutes using `.summary()`. For better visualization, we can feed the summary into a pandas DataFrame.

In [10]:
df = pd.DataFrame(snaps.summary())
print("Shape:", df.shape)
df.head(10)

Shape: (48, 102)


,level_0_frame10,timeEnemySpentControlled_0_frame10,totalGold_0_frame10,xp_0_frame10,totalDamageDone_0_frame10,totalDamageDoneToChampions_0_frame10,totalDamageTaken_0_frame10,creepScore_0_frame10,goldPorportion_0_frame10,xpPorportion_0_frame10,...,totalGold_4_frame15,xp_4_frame15,totalDamageDone_4_frame15,totalDamageDoneToChampions_4_frame15,totalDamageTaken_4_frame15,creepScore_4_frame15,goldPorportion_4_frame15,xpPorportion_4_frame15,matchId,win
0,1,38572,923,1358,11320,2128,1676,4,0.063357,0.081517,...,-963,332,-19775,-2245,-37,-36,-0.031478,0.021289,KR_6004304384,False
1,0,131400,-927,98,17737,-4158,2904,-8,-0.047990,-0.001546,...,3582,-166,25860,4722,-891,52,0.119096,-0.000790,KR_6004350323,False
2,-1,-76507,-3074,-630,-15294,-1845,-1290,2,-0.108747,0.005951,...,-1771,-2826,-23942,164,913,-32,0.007948,-0.026234,KR_6004238576,False
3,1,128361,1652,897,11816,8224,-2299,3,0.033643,0.013746,...,1668,1669,1031,9004,-622,-11,0.021866,0.013260,KR_6004246112,True
4,0,124556,-1141,200,11636,57,1376,15,-0.043561,-0.005800,...,241,162,6441,5093,685,-18,-0.003641,0.003451,KR_6004223188,True
5,-1,-189578,-433,-218,7815,8628,-3480,3,-0.016313,-0.012386,...,-1526,-292,2582,641,-8729,1,-0.019690,0.007293,KR_6004159593,False
6,0,-173860,802,568,-1641,-123,1289,-5,0.030040,0.009728,...,-3969,467,-47314,-12931,6850,-50,-0.073015,-0.002268,KR_6001228140,False
7,0,24082,855,55,-1556,6312,2200,-22,0.020868,-0.007431,...,-614,-494,24296,8092,-15813,3,-0.021030,-0.028651,KR_5997217063,True
8,1,8741,-125,1148,-274,4979,675,-14,0.002593,0.037620,...,2483,182,25375,5317,-4773,27,0.050240,0.004607,KR_5997180469,False
9,1,161381,-301,1011,6076,2533,-619,11,-0.012352,0.014554,...,-3075,908,14741,-7101,-5314,32,-0.062518,0.001200,KR_5997056717,True


## Features

Each number represents the **difference** between the blue side and the red side. The column names might be very confusing at first, but you will understand it if you know how the columns are named. 

The columns names follow this pattern: _feature\_lane#\_frame#_. 

For example: _level\_0\_frame10_ represents the **difference in level between the 0th lane (TOP lane) player at the 10 minute mark**. For conversion of the lane code, please refer:

    {0:"TOP", 1:"JUG", 2:"MID", 3:"BOT", 4:"SUP"}

## What's Next?

Data from all the matches are now in an organized 2 dimension tabular format. From here, we can feed the data to your favorite machine learning models. 

You can do more manipulation using the `SnapShots` class. For example, if you are familiar with League of Legends, you can sepcify the **lanes** you are interested in

In [11]:
snaps.get_lanes(["TOP", "MID"], per_frame=False)

ValueError: You must specify per_frame to either `True` or `False`.